In [2]:
import json
from collections import defaultdict
import copy
import pandas as pd

In [3]:
data = json.load(open('/root/EarleeNLP/data/datafountain/train.json', 'r'))
data

{'NLU00000': {'text': '还有双鸭山到淮阴的汽车票吗13号的',
  'intent': 'Travel-Query',
  'slots': {'destination': '淮阴',
   'departure': '双鸭山',
   'datetime_date': '13号',
   'query_type': '汽车票'}},
 'NLU00001': {'text': '从这里怎么回家',
  'intent': 'Travel-Query',
  'slots': {'destination': '家', 'query_type': '导航'}},
 'NLU00002': {'text': '随便播放一首专辑阁楼里的佛里的歌',
  'intent': 'Music-Play',
  'slots': {'album': '阁楼', 'play_mode': '随机播放'}},
 'NLU00003': {'text': '给看一下墓王之王嘛',
  'intent': 'FilmTele-Play',
  'slots': {'name': '墓王之王'}},
 'NLU00004': {'text': '我想看挑战两把s686打突变团竞的游戏视频',
  'intent': 'Video-Play',
  'slots': {'name': '挑战两把s686打突变团竞的游戏视频'}},
 'NLU00005': {'text': '我想看和平精英上战神必备技巧的游戏视频',
  'intent': 'Video-Play',
  'slots': {'name': '和平精英上战神必备技巧的游戏视频'}},
 'NLU00006': {'text': '2019年古装爱情电视剧小女花不弃的花絮播放一下',
  'intent': 'Video-Play',
  'slots': {'name': '小女花不弃的花絮', 'datetime_date': '2019年'}},
 'NLU00007': {'text': '找一个2004年的推理剧给我看一会呢',
  'intent': 'FilmTele-Play',
  'slots': {'tag': '推理', 'age': '2004年'}},
 'NLU00008'

In [4]:
def normalization(text):
    candidate_num_list = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    candidate_letter_list = [
        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
        'v', 'w', 'x', 'y', 'z',
        'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U',
        'V', 'W', 'X', 'Y', 'Z'
    ]
    for i in candidate_num_list:
        text = text.replace(i, '*')
        
    for i in candidate_letter_list:
        text = text.replace(i, '@')
        
    return text

In [19]:
# BIO 
nested_ner_num = 0
nested_ner_idx = []

seq_normalization_text = []
seq_text = []
seq_label = []
seq_intent = []
seq_idx = []
for idx, info_dict in data.items():
    intent = info_dict['intent']
    text = info_dict['text']
    slots = info_dict['slots']
    if idx in ['NLU00247', 'NLU00149', 'NLU00247', 'NLU00294', 'NLU00479', 'NLU00722', 'NLU01044' \
              ,'NLU01203', 'NLU01233', 'NLU01344', 'NLU02386', 'NLU03108', 'NLU03302', 'NLU03829', \
              'NLU04934', 'NLU05234', 'NLU05801', 'NLU06413', 'NLU07935', 'NLU08272', 'NLU08449', \
              'NLU08552', 'NLU08880']:
        continue
    if text == '你好李焕英有了吧赶紧播放':
        continue
    default_label = len(text) * ['O']
    feasible_mark = True
    for slot_k, slot_value in slots.items():
        
        if slot_k in ['query_type', 'play_mode', 'command', 'index', 'language', 'type']:
            continue
            
            
        if slot_value == '银临，河图':
            slot_value = ['银临', '河图']
        elif slot_value == '那英，田震':
            slot_value = ['那英', '田震']
        elif slot_value == '谷枫，菲菲':
            slot_value = ['谷枫', '菲菲']
        elif slot_value == '姐姐 孤独的人是可耻的 蚂蚁':
            slot_value = ['姐姐', '孤独的人是可耻的', '蚂蚁']
        elif slot_value == '花粥，马玉阳':
            slot_value = ['花粥', '马玉阳']
        elif slot_value == '银临，云泣':
            slot_value = ['银临', '云泣']
        elif slot_value == '余生 小半':
            slot_value = ['余生', '小半']
        elif slot_value == '堕落天使 蕾丝花边':
            slot_value = ['堕落天使', '蕾丝花边']
        elif slot_value == '80年代 ':
            slot_value = '80年代'
        elif slot_value == '江伟麟，陈嘉慧':
            slot_value = ['江伟麟', '陈嘉慧']
        elif slot_value == '90年代 ':
            slot_value = '90年代'
        elif slot_value == '花粥，王胜男':
            slot_value = ['花粥', '王胜男']
        elif slot_value == '王菲，那英':
            slot_value = ['王菲', '那英']
        elif slot_value == '河图，洛天依':
            slot_value = ['河图', '洛天依']
        elif slot_value == '陈伟霆 ':
            slot_value = '陈伟霆'
        elif slot_value == '黄诗扶，B2ER':
            slot_value = ['黄诗扶', 'B2ER']
        elif text == '播放一个韩剧窥探吧' and slot_value == '韩国':
            slot_value = '韩剧'
        elif text == '给我找一个美剧偶像剧播放一下吧' and slot_value == '美国':
            slot_value = '美剧'
        elif text == '港片新喜剧之王播放一下可以吗' and slot_value == '香港':
            slot_value = '港片'
        elif text == '能找一个范迪塞尔导演的美剧看看么' and slot_value == '美国':
            slot_value = '美剧'
        elif text == '你能找一个范迪塞尔拍摄的美剧给我看吗' and slot_value == '美国':
            slot_value = '美剧'
        elif text == '给我播放韩剧宋仲基主演的太阳的后裔。' and slot_value == '韩国':
            slot_value = '韩剧'
        elif text == '我想看一会港片新喜剧之王呢。' and slot_value == '香港':
            slot_value = '港片'
        elif text == '我想看一会港片新喜剧之王呢' and slot_value == '香港':
            slot_value = '港片'
        elif text == '泰剧禁忌女孩可以播放一下不' and slot_value == '泰国':
            slot_value = '泰剧'
        elif text == '我想看一会韩剧，那个朴信惠出演的可以吗' and slot_value == '韩国':
            slot_value = '韩剧'
        elif text == '秋瓷炫演出的韩剧给我播放一个' and slot_value == '韩国':
            slot_value = '韩剧'
        elif text == '范迪塞尔导演的美剧可以来一个看看不' and slot_value == '美国':
            slot_value = '美剧'
        elif text == '张国荣的港片色情男女播放吧' and slot_value == '香港':
            slot_value = '港片'
        elif text == '我想看一个比较轻松快乐的美剧' and slot_value == '美国':
            slot_value = '美剧'
        elif text == '你能找一个内地拍的神话剧看会吗古代的特效好点的' and slot_value == '大陆':
            slot_value = '内地'
        elif text == '美剧2009年上线的播放一下' and slot_value == '美国':
            slot_value = '美剧'
        elif text == '美片指环王2播放一下给我看' and slot_value == '美国':
            slot_value = '美片'
        elif text == '你能给我播放一下韩剧W两个世界吗' and slot_value == '韩国':
            slot_value = '韩剧'
        elif text == '韩剧顶楼可以播放出来吗' and slot_value == '韩国':
            slot_value = '韩剧'
        elif text == '内地剧一起同过窗带有人生哲理，让我感受一下吧' and slot_value == '大陆':
            slot_value = '内地'
        elif text == '能来一个安妮海瑟薇的美剧看看吗' and slot_value == '美国':
            slot_value = '美剧'
        elif text == '张根硕的韩剧可以找一个看不' and slot_value == '韩国':
            slot_value = '韩剧'
        elif text == '是不是能找一个2018年的美剧看一下' and slot_value == '美国':
            slot_value = '美剧'
        elif text == '我想看美剧悬疑片了就后裔' and slot_value == '美国':
            slot_value = '美剧'
        elif text == '快点帮我播放韩剧请回答1988' and slot_value == '韩国':
            slot_value = '韩剧'
        elif text == '能不能播放韩剧太阳的后裔给我看啊' and slot_value == '韩国':
            slot_value = '韩剧'
        elif text == '罗晋唐嫣演的宫廷剧锦绣未央快放一下' and slot_value == ['唐嫣', '罗晋 ']:
            slot_value = ['唐嫣', '罗晋']
        elif text == '播放NBA全明星赛给我 ':
            text = '播放NBA全明星赛给我'


        if isinstance(slot_value, str):
            slot_value = [slot_value]
        for v in slot_value:
            try:
                start_idx = text.index(v)
            except Exception as e:
                print(slot_k)
                print(slot_value)
                print('=====')
                print(v)
                print(text)
                raise e
            if len(v) == 1:
                if default_label[start_idx] != 'O':
                    nested_ner_num += 1
                    nested_ner_idx.append(idx)
                    feasible_mark = False
                    break
                else:
#                     default_label[start_idx] = 'B-' + str(intent) + '-' + str(slot_k)
                    default_label[start_idx] = 'B-' + str(slot_k)

            else:
                if default_label[start_idx] != 'O':
                    nested_ner_num += 1
                    nested_ner_idx.append(idx)
                    feasible_mark = False
                    break
                else:
#                     default_label[start_idx] = 'B-' + str(intent) + '-' + str(slot_k)
                    default_label[start_idx] = 'B-' + str(slot_k)
                for extra_idx, char in enumerate(v[1:]):
                    if default_label[start_idx + 1 + extra_idx] != 'O':
                        nested_ner_num += 1
                        nested_ner_idx.append(idx)
                        feasible_mark = False
                        break
                    else:
#                         default_label[start_idx + 1 + extra_idx] = 'I-' + str(intent) + '-' + str(slot_k)
                        default_label[start_idx + 1 + extra_idx] = 'I-' + str(slot_k)

    if feasible_mark is True:
        seq_normalization_text.append(normalization(text))
        seq_text.append(text)
        seq_label.append(default_label)
        seq_intent.append(intent2idx[intent])
        seq_idx.append([label2idx[ll.split('-')[0]+'-'+intent+'-'+ll.split('-')[-1]] if ll != 'O' else label2idx[ll] for ll in default_label])

In [20]:
seq4df = {
    'text': seq_normalization_text,
    'seq_label': seq_label,
    'seq_idx': seq_idx,
    'seq_intent': seq_intent
}
df = pd.DataFrame(seq4df)
df

,text,seq_label,seq_idx,seq_intent
0,还有双鸭山到淮阴的汽车票吗**号的,"[O, O, B-departure, I-departure, I-departure, ...","[0, 0, 42, 38, 38, 0, 18, 10, 0, 0, 0, 0, 0, 5...",2
1,从这里怎么回家,"[O, O, O, O, O, O, B-destination]","[0, 0, 0, 0, 0, 0, 18]",2
2,随便播放一首专辑阁楼里的佛里的歌,"[O, O, O, O, O, O, O, O, B-album, I-album, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 58, 35, 0, 0, 0, 0, 0...",0
3,给看一下墓王之王嘛,"[O, O, O, O, B-name, I-name, I-name, I-name, O]","[0, 0, 0, 0, 29, 5, 5, 5, 0]",4
4,我想看挑战两把@***打突变团竞的游戏视频,"[O, O, O, B-name, I-name, I-name, I-name, I-na...","[0, 0, 0, 23, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",6
...,...,...,...,...
8922,播放周深的英文歌,"[O, O, B-artist, I-artist, O, O, O, O]","[0, 0, 36, 20, 0, 0, 0, 0]",0
8923,我想知道开原今天是不是要多加衣物防寒保暖。,"[O, O, O, O, B-city, I-city, B-datetime_date, ...","[0, 0, 0, 0, 22, 16, 24, 14, 0, 0, 0, 0, 0, 0,...",5
8924,搜索*月**日去阿姆斯特丹的经济舱价格。,"[O, O, B-datetime_date, I-datetime_date, I-dat...","[0, 0, 50, 41, 41, 41, 41, 0, 18, 10, 10, 10, ...",2
8925,给我看看北京飞上海的飞机已经起飞了吗,"[O, O, O, O, B-departure, I-departure, O, B-de...","[0, 0, 0, 0, 42, 38, 0, 18, 10, 0, 0, 0, 0, 0,...",2


In [21]:
df.to_pickle('/ai/223/person/lichunyu/datasets/dataf/seq_label/seq_label_8927.pkl')

In [8]:
INTENT = ['Music-Play',
        'HomeAppliance-Control',
        'Travel-Query',
        'Calendar-Query',
        'FilmTele-Play',
        'Weather-Query',
        'Video-Play',
        'Alarm-Update',
        'TVProgram-Play',
        'Audio-Play',
        'Radio-Listen',
        'Other']

intent2idx = {i: idx for idx, i in enumerate(INTENT)}
intent2idx

{'Music-Play': 0,
 'HomeAppliance-Control': 1,
 'Travel-Query': 2,
 'Calendar-Query': 3,
 'FilmTele-Play': 4,
 'Weather-Query': 5,
 'Video-Play': 6,
 'Alarm-Update': 7,
 'TVProgram-Play': 8,
 'Audio-Play': 9,
 'Radio-Listen': 10,
 'Other': 11}

In [6]:
train_seq_normalization_text = seq_normalization_text[:7401]
train_seq_label = seq_label[:7401]

test_seq_normalization_text = seq_normalization_text[7401:]
test_seq_label = seq_label[7401:]

In [7]:
train_seq_normalization_text

['还有双鸭山到淮阴的汽车票吗**号的',
 '从这里怎么回家',
 '随便播放一首专辑阁楼里的佛里的歌',
 '给看一下墓王之王嘛',
 '我想看挑战两把@***打突变团竞的游戏视频',
 '我想看和平精英上战神必备技巧的游戏视频',
 '****年古装爱情电视剧小女花不弃的花絮播放一下',
 '找一个****年的推理剧给我看一会呢',
 '自驾游去深圳都经过那些地方啊',
 '给我转播今天的女子双打乒乓球比赛现场',
 '单曲循环一首催眠的歌吧',
 '你能播放一下曹盾导演的古装悬疑剧呢',
 '河南新闻广播找一下啊是新闻台',
 '交通类的武汉交通广播电台来一个吧',
 '播放香港电台的王菲的经典音乐',
 '给我找一个魔兽世界的比赛视频',
 '播放钢琴曲命运交响曲',
 '早上七点开空调',
 '查询北京飞桂林的飞机是否已经起飞了',
 '播放中央电台里的都市之声',
 '客厅空调风速低一些低一些',
 '海南今天几级风',
 '打开烤箱的开关按钮',
 '帮我定一个**号下午*点参加公司年会的闹钟。',
 '儿童房的空调调高*度',
 '*月**号零点京东的拍卖活动提醒我一下。',
 '能随机给我来一个赵丽颖****年上的剧吗',
 '麻烦帮我创建一个周末上午*点和女朋友的约会提醒。',
 '我要听治愈系的日语歌曲',
 '我一定要单曲循环赵雷的我们的时光这首流行',
 '*月*号下午*点之前的提醒取消掉',
 '让洗衣机停止工作好吗',
 '昨天秦岭是不是刮风了',
 '我要出去旅游几天，帮我把灯设置为离家模式吧',
 '音乐老友记拨给我是山西音乐广播的',
 '我想看****年谍战电视剧秋蝉第三集花絮',
 '能把家庭剧小欢喜给我播放一下吗',
 '五一劳动节是农历几时',
 '杨延东****年上线的剧播放来看看呢',
 '播放美国推理电影蜘蛛侠',
 '快点帮我定一个周末上午*点跟家人一起去动物园的闹钟。',
 '把盛志耘主持的神州之声播放一下呢',
 '快帮我查一下明天是星期几',
 '可不可以把电饭煲开到蛋糕模式呢',
 '查询衡水晚上气温多少',
 '订三张明天早上飞成都的经济舱',
 '农历的*月**是几号',
 '下周六我爷爷让我去买茶叶，记得提醒我',
 '明天叫我去森林野炊',
 '龙抬头是什么时候',
 '明天*点的闹钟不要再响了'

In [60]:
with open('/ai/223/person/lichunyu/datasets/dataf/seq_label/seq_label_non_intent.train', 'w') as f:
    for idx in range(len(train_seq_normalization_text)):
        for char_idx in range(len(train_seq_normalization_text[idx])):
                if train_seq_normalization_text[idx][char_idx] and train_seq_normalization_text[idx][char_idx] != ' ':
                    f.write(train_seq_normalization_text[idx][char_idx] + '\t' + train_seq_label[idx][char_idx] + '\n')
        f.write('\n')

In [61]:
with open('/ai/223/person/lichunyu/datasets/dataf/seq_label/seq_label_non_intent.test', 'w') as f:
    for idx in range(len(test_seq_normalization_text)):
        for char_idx in range(len(test_seq_normalization_text[idx])):
            if test_seq_normalization_text[idx][char_idx] and test_seq_normalization_text[idx][char_idx] != ' ':
                f.write(test_seq_normalization_text[idx][char_idx] + '\t' + test_seq_label[idx][char_idx] + '\n')

        f.write('\n')

In [32]:
train_seq_normalization_text

['还有双鸭山到淮阴的汽车票吗**号的',
 '从这里怎么回家',
 '随便播放一首专辑阁楼里的佛里的歌',
 '给看一下墓王之王嘛',
 '我想看挑战两把#***打突变团竞的游戏视频',
 '我想看和平精英上战神必备技巧的游戏视频',
 '****年古装爱情电视剧小女花不弃的花絮播放一下',
 '找一个****年的推理剧给我看一会呢',
 '自驾游去深圳都经过那些地方啊',
 '给我转播今天的女子双打乒乓球比赛现场',
 '单曲循环一首催眠的歌吧',
 '你能播放一下曹盾导演的古装悬疑剧呢',
 '河南新闻广播找一下啊是新闻台',
 '交通类的武汉交通广播电台来一个吧',
 '播放香港电台的王菲的经典音乐',
 '给我找一个魔兽世界的比赛视频',
 '播放钢琴曲命运交响曲',
 '早上七点开空调',
 '查询北京飞桂林的飞机是否已经起飞了',
 '播放中央电台里的都市之声',
 '客厅空调风速低一些低一些',
 '海南今天几级风',
 '打开烤箱的开关按钮',
 '帮我定一个**号下午*点参加公司年会的闹钟。',
 '儿童房的空调调高*度',
 '*月**号零点京东的拍卖活动提醒我一下。',
 '能随机给我来一个赵丽颖****年上的剧吗',
 '麻烦帮我创建一个周末上午*点和女朋友的约会提醒。',
 '我要听治愈系的日语歌曲',
 '我一定要单曲循环赵雷的我们的时光这首流行',
 '*月*号下午*点之前的提醒取消掉',
 '让洗衣机停止工作好吗',
 '昨天秦岭是不是刮风了',
 '我要出去旅游几天，帮我把灯设置为离家模式吧',
 '音乐老友记拨给我是山西音乐广播的',
 '我想看****年谍战电视剧秋蝉第三集花絮',
 '能把家庭剧小欢喜给我播放一下吗',
 '五一劳动节是农历几时',
 '杨延东****年上线的剧播放来看看呢',
 '播放美国推理电影蜘蛛侠',
 '快点帮我定一个周末上午*点跟家人一起去动物园的闹钟。',
 '把盛志耘主持的神州之声播放一下呢',
 '快帮我查一下明天是星期几',
 '可不可以把电饭煲开到蛋糕模式呢',
 '查询衡水晚上气温多少',
 '订三张明天早上飞成都的经济舱',
 '农历的*月**是几号',
 '下周六我爷爷让我去买茶叶，记得提醒我',
 '明天叫我去森林野炊',
 '龙抬头是什么时候',
 '明天*点的闹钟不要再响了'

In [14]:
IDX2LABEL = {
    0: '<pad>', 
    1: '<unk>', 
    2: 'O', 
    3: 'I-Video-Play-name', 
    4: 'I-Radio-Listen-channel', 
    5: 'I-Calendar-Query-datetime_date', 
    6: 'I-Alarm-Update-notes', 
    7: 'I-FilmTele-Play-name', 
    8: 'I-Alarm-Update-datetime_time', 
    9: 'I-Radio-Listen-name', 
    10: 'I-Alarm-Update-datetime_date', 
    11: 'I-HomeAppliance-Control-appliance', 
    12: 'I-Travel-Query-destination', 
    13: 'I-HomeAppliance-Control-details', 
    14: 'I-Radio-Listen-frequency', 
    15: 'I-Music-Play-song', 
    16: 'I-Weather-Query-datetime_date', 
    17: 'B-Calendar-Query-datetime_date', 
    18: 'I-Weather-Query-city', 
    19: 'B-HomeAppliance-Control-appliance', 
    20: 'B-Travel-Query-destination', 
    21: 'I-Video-Play-datetime_date', 
    22: 'I-Music-Play-artist', 
    23: 'B-Alarm-Update-datetime_date', 
    24: 'B-Weather-Query-city', 
    25: 'B-Video-Play-name', 
    26: 'B-Weather-Query-datetime_date', 
    27: 'B-Alarm-Update-datetime_time', 
    28: 'I-FilmTele-Play-artist', 
    29: 'B-Alarm-Update-notes', 
    30: 'B-HomeAppliance-Control-details', 
    31: 'B-FilmTele-Play-name', 
    32: 'B-Radio-Listen-channel', 
    33: 'I-Music-Play-age', 
    34: 'I-FilmTele-Play-age', 
    35: 'B-Radio-Listen-name', 
    36: 'I-FilmTele-Play-tag', 
    37: 'I-Music-Play-album', 
    38: 'B-Music-Play-artist', 
    39: 'B-FilmTele-Play-artist', 
    40: 'I-Travel-Query-departure', 
    41: 'B-Music-Play-song', 
    42: 'I-FilmTele-Play-play_setting', 
    43: 'I-Travel-Query-datetime_date', 
    44: 'B-Travel-Query-departure', 
    45: 'I-Radio-Listen-artist', 
    46: 'B-FilmTele-Play-tag', 
    47: 'I-Travel-Query-datetime_time', 
    48: 'B-Radio-Listen-frequency', 
    49: 'B-Radio-Listen-artist', 
    50: 'I-Video-Play-datetime_time', 
    51: 'B-Video-Play-datetime_date', 
    52: 'B-Travel-Query-datetime_date', 
    53: 'I-FilmTele-Play-region', 
    54: 'B-FilmTele-Play-region', 
    55: 'B-FilmTele-Play-play_setting', 
    56: 'I-TVProgram-Play-name', 
    57: 'B-FilmTele-Play-age', 
    58: 'B-Travel-Query-datetime_time', 
    59: 'B-Music-Play-age', 
    60: 'B-Music-Play-album', 
    61: 'I-Video-Play-region', 
    62: 'B-Video-Play-region', 
    63: 'I-Music-Play-instrument', 
    64: 'I-Weather-Query-datetime_time', 
    65: 'I-TVProgram-Play-channel', 
    66: 'B-Music-Play-instrument', 
    67: 'I-Audio-Play-name', 
    68: 'B-Video-Play-datetime_time', 
    69: 'B-Weather-Query-datetime_time', 
    70: 'B-TVProgram-Play-name', 
    71: 'I-TVProgram-Play-datetime_date', 
    72: 'I-Audio-Play-artist', 
    73: 'B-Audio-Play-name', 
    74: 'I-TVProgram-Play-datetime_time', 
    75: 'B-TVProgram-Play-channel', 
    76: 'B-TVProgram-Play-datetime_date', 
    77: 'B-Audio-Play-artist', 
    78: 'B-TVProgram-Play-datetime_time', 
    79: 'I-Audio-Play-play_setting', 
    80: 'B-Audio-Play-play_setting', 
    81: 'B-Audio-Play-tag', 
    82: 'I-Audio-Play-tag'
}

label2idx = {v: k-2 for k, v in IDX2LABEL.items() if v not in ['<pad>', '<unk>']}
label2idx

{'O': 0,
 'I-Video-Play-name': 1,
 'I-Radio-Listen-channel': 2,
 'I-Calendar-Query-datetime_date': 3,
 'I-Alarm-Update-notes': 4,
 'I-FilmTele-Play-name': 5,
 'I-Alarm-Update-datetime_time': 6,
 'I-Radio-Listen-name': 7,
 'I-Alarm-Update-datetime_date': 8,
 'I-HomeAppliance-Control-appliance': 9,
 'I-Travel-Query-destination': 10,
 'I-HomeAppliance-Control-details': 11,
 'I-Radio-Listen-frequency': 12,
 'I-Music-Play-song': 13,
 'I-Weather-Query-datetime_date': 14,
 'B-Calendar-Query-datetime_date': 15,
 'I-Weather-Query-city': 16,
 'B-HomeAppliance-Control-appliance': 17,
 'B-Travel-Query-destination': 18,
 'I-Video-Play-datetime_date': 19,
 'I-Music-Play-artist': 20,
 'B-Alarm-Update-datetime_date': 21,
 'B-Weather-Query-city': 22,
 'B-Video-Play-name': 23,
 'B-Weather-Query-datetime_date': 24,
 'B-Alarm-Update-datetime_time': 25,
 'I-FilmTele-Play-artist': 26,
 'B-Alarm-Update-notes': 27,
 'B-HomeAppliance-Control-details': 28,
 'B-FilmTele-Play-name': 29,
 'B-Radio-Listen-channel':

In [4]:
_correspondence = defaultdict(list)
for idx, info_dict in data.items():
    if info_dict['slots']:
        _correspondence[info_dict['intent']].extend(list(info_dict['slots'].keys()))

correspondence = defaultdict(list)
for k, v in _correspondence.items():
    correspondence[k] = list(set(v))
    
correspondence

defaultdict(list,
            {'Travel-Query': ['datetime_time',
              'datetime_date',
              'destination',
              'query_type',
              'departure'],
             'Music-Play': ['instrument',
              'play_mode',
              'song',
              'artist',
              'language',
              'album',
              'age'],
             'FilmTele-Play': ['name',
              'artist',
              'tag',
              'play_setting',
              'age',
              'region'],
             'Video-Play': ['datetime_time',
              'name',
              'region',
              'datetime_date'],
             'Radio-Listen': ['channel', 'name', 'artist', 'frequency'],
             'HomeAppliance-Control': ['details', 'appliance', 'command'],
             'Weather-Query': ['city',
              'datetime_date',
              'type',
              'index',
              'datetime_time'],
             'Alarm-Update': ['datetime_time', 'datetim